In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path().cwd().parent))

In [3]:
from typing import Tuple

from itertools import product

import pandas as pd

from plotting import plot_ts
from load_dataset import Dataset
from model import TimeSeriesPredictor

/Users/g.sinyakov/ts_winter/model.py:233: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def predict_batch(self, ts: pd.Series, ts_batch: pd.Series = pd.Series()):


In [41]:
import warnings

warnings.filterwarnings(action='ignore')

### Какие ряды будем тестировать?

* длинный ряд с сезонностью  
* короткий ряд с сезонностью  
* короткий ряд с сезонностью и трендом  
* случайное блуждание  
* средне зашумленный ряд
* "шумный" ряд  

In [42]:
ds = Dataset('../data/dataset/')

In [43]:
long = ds['daily-min-temperatures.csv']

In [44]:
plot_ts(long)

Loading BokehJS ...

In [45]:
short_season = ds['hour_3019.csv'][300:]

In [46]:
plot_ts(short_season)

Loading BokehJS ...

In [48]:
short_season_trend = ds['international-airline-passengers.csv']

In [49]:
plot_ts(short_season_trend)

Loading BokehJS ...

In [50]:
random_walk = ds['dow_jones_0.csv']

In [51]:
plot_ts(random_walk)

Loading BokehJS ...

In [52]:
medium_noize = ds['hour_3426.csv'][300:]

In [53]:
plot_ts(medium_noize)

Loading BokehJS ...

In [54]:
full_noize = ds['day_1574.csv']

In [55]:
plot_ts(full_noize)

Loading BokehJS ...

### Какие модели будем тестировать?

* скользящее среднее
* экспоненциальное сглаживание
* autoArima
* линейная регрессия
* линейная регрессия с L1 регуляризацией (Lasso)
* RandomForeset
* градиентный бустинг


In [70]:
from estimators import RollingEstimator, ExponentialSmoothingEstimator
from pmdarima import auto_arima
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

### По каким метрикам будем сравнивать?

* mse
* mae
* R2
* mase

In [57]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score

from metrics import mean_absolute_percentage_error as mape
from metrics import mase

### По какой методике будем тестировать?

* 70% трейн, 30% тест
* Out-of-sample, чтобы посмотреть как модель предсказывает "вдолгую"
* In-Sample, чтобы посмотреть как модель предсказывает на одну точку вперед
* Для поиска гиперпараметров можно делать кроссвалидацию на тесте по метрике mse

### Задание 1. Напишите функцию, разбивающую на train и test

In [58]:
def train_test_split(ts: pd.Series, ratio: float = 0.7) -> Tuple[pd.Series]:
    split_idx = int(len(ts) * ratio)
    ts_train, ts_test = ts[:split_idx], ts[split_idx:]
    # ваш код здесь
    return ts_train, ts_test

### Зададим соответствие гранулярностей для наших рядов.

In [59]:
granularity_mapping = {
    'long': 'P1D',
    'short_season': 'PT1H',
    'short_season_trend': 'P1M',
    'random_walk': 'P1D',
    'medium_noize': 'PT1H',
    'full_noize': 'P1D'
}

### Зададим обязательные datetime признаки

In [76]:
import math

from pandas._libs.tslibs.timestamps import Timestamp


def get_month_sin(timestamp: Timestamp) -> float:
    theta = timestamp.month * (2*math.pi / 12)
    return math.sin(theta)


def get_month_cos(timestamp: Timestamp) -> float:
    theta = timestamp.month * (2*math.pi / 12)
    return math.cos(theta)


def get_day_sin(timestamp: Timestamp) -> float:
    theta = timestamp.day * (2*math.pi / timestamp.days_in_month)
    return math.sin(theta)


def get_day_cos(timestamp: Timestamp) -> float:
    theta = timestamp.day * (2*math.pi / timestamp.days_in_month)
    return math.cos(theta)


def get_dayofweek_sin(timestamp: Timestamp) -> float:
    theta = timestamp.dayofweek * (2*math.pi / 7)
    return math.sin(theta)


def get_dayofweek_cos(timestamp: Timestamp) -> float:
    theta = timestamp.dayofweek * (2*math.pi / 7)
    return math.cos(theta)


def get_hour_sin(timestamp: Timestamp) -> float:
    theta = timestamp.hour * (2*math.pi / 24)
    return math.sin(theta)


def get_hour_cos(timestamp: Timestamp) -> float:
    theta = timestamp.hour * (2*math.pi / 24)
    return math.cos(theta)


def get_minute_sin(timestamp: Timestamp) -> float:
    theta = timestamp.minute * (2*math.pi / 60)
    return math.sin(theta)


def get_minute_cos(timestamp: Timestamp) -> float:
    theta = timestamp.minute * (2*math.pi / 60)
    return math.cos(theta)


datetime_mappers = {
    'month_sin': get_month_sin,
    'month_cos': get_month_cos,
    'day_sin': get_day_sin,
    'day_cos': get_day_cos,
    'dayofweek_sin': get_dayofweek_sin,
    'dayofweek_cos': get_dayofweek_cos,
    'hour_sin': get_hour_sin,
    'hour_cos': get_hour_cos,
    'minute_sin': get_minute_sin,
    'minute_cos': get_minute_cos,
}

### Задание 2. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через TimeSeriesPredictor.

* принмает на вход исходный ряд, гранулярность, количество лагов, модель, а также **kwargs, в которые мы будем передавать параметры модели

* разбивает ряд на train/test

* создает инстанс TimeSeriesPredictor с нужными параметрами

* обучает предиктор на трейне

* делает out_of_sample и in_sample прогноз

* возвращает train, test, in_sample, out_of_sample

In [94]:
def make_pipeline(
    ts: pd.Series,
    granularity: str,
    model: callable,
    num_lags=24,
    use_mappers=True,
    **kwargs
) -> Tuple[pd.Series]:
    
    train, test = train_test_split(ts)
    
    predictor = TimeSeriesPredictor(
        granularity=granularity,
        num_lags=num_lags,
        model=model,
    )
    
    if use_mappers:
        predictor.set_params(mappers=datetime_mappers)
        
    predictor.set_params(**kwargs)
    predictor.fit(train)
    
    in_sample = predictor.predict_batch(train, test)
    horizon = 
    out_of_sample = predictor.predict_next(train, len(test))
    
    ## temp bug fix, when test has missing values
    out_of_sample.index = test.index
    
    return train, test, in_sample, out_of_sample

In [110]:
def hyperparameters_search(ts, granularity, model, num_lags, param_grid, verbose=False, use_mappers=True):
    
    statistics_in_sample, statistics_out_of_sample = {}, {}
    
    for param_tuple in product(*param_grid.values()):
        params = dict(zip(param_grid.keys(), param_tuple))
        
        num_lags = params.pop('num_lags', None) or num_lags
        train, test, in_sample, out_of_sample = make_pipeline(
            ts, granularity, model, num_lags, use_mappers=use_mappers, **params)
        
        mse_in_sample = mse(test, in_sample)
        mae_in_sample = mae(test, in_sample)
        r2_score_in_sample = r2_score(test, in_sample)
        mase_in_sample = mase(in_sample, test)
        
        mse_out_of_sample = mse(test, out_of_sample)
        mae_out_of_sample = mae(test, out_of_sample)
        r2_score_out_of_sample = r2_score(test, out_of_sample)
        mase_out_of_sample = mase(out_of_sample, test)
        
        statistics_in_sample[param_tuple] = mse_in_sample
        statistics_out_of_sample[param_tuple] = mse_out_of_sample
        

        if verbose:
            print(f'Params are: {params}')
            print(
                f"""
                IN_SAMPLE
                mse: {mse(test, in_sample)},
                mae: {mae(test, in_sample)},
                r2: {r2_score(test, in_sample)},
                mase: {mase(in_sample, test)}
                ---------------------
                OUT_OF_SAMPLE
                mse: {mse(test, out_of_sample)},
                mae: {mae(test, out_of_sample)},
                r2: {r2_score(test, out_of_sample)},
                mase: {mase(out_of_sample, test)}
                """
            )
            plot_ts(train, test, in_sample, out_of_sample)
            
    
    best_in_sample = sorted(statistics_in_sample.items(), key=lambda x: x[1])[0]
    best_out_of_sample = sorted(statistics_out_of_sample.items(), key=lambda x: x[1])[0]
    
    best_in_sample = dict(zip(param_grid.keys(), best_in_sample[0]))
    best_out_of_sample = dict(zip(param_grid.keys(), best_out_of_sample[0]))
    
    
    if verbose:
        print(f"best in_sample params are {best_in_sample}")
        print(f"best out_of_sample params are {best_out_of_sample}")
    
    return best_in_sample, best_out_of_sample

### Задание 3. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через auto_arima

* функция должна принимать исходный временной ряд, период сезонности, параметры дифференцирования d, D и boolean параметр seasonal, данные параметры будут являться для нас гиперпараметрами, все остальное за нас должна найти auto_arima

* разбивает на train, test

* обучает arima на train при помощи вызова функции auto_arima из библиотеки pmdarima с переданными параметрами и со следующими зафиксированными параметрами: `max_p=3, max_q=3, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True`

* в качестве out_of_sample прогноза просто вызовите метод predict

* в качестве in_sample прогноза обучите модель заново на всём ряде методом `fit`, вызовите метод predict_in_sample и в качестве прогноза возьмите `in_sample_predictions(-len(test):)`

* возвращает train, test, in_sample, out_of_sample (не забудьте сделать их pd.Series с нужным индексом!!)

In [111]:
def make_pipeline_arima(ts: pd.Series, period: int, d: int = 1, D: int = 1, seasonal: bool = True) -> Tuple[pd.Series]:
    train, test = train_test_split(ts)
    
    arima_fit = auto_arima(
        train,
        max_p=3, max_q=3, m=period,
        seasonal=seasonal,
        d=d, D=D,
        trace=True,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True
    )
    
    out_of_sample = pd.Series(data=arima_fit.predict(len(test)), index=test.index)
    
    arima_fit.fit(ts)
    
    in_sample = pd.Series(arima_fit.predict_in_sample()[-len(test):], index=test.index)
    
    return train, test, in_sample, out_of_sample

### Задание 4. "Прогоните" все алгоритмы на всех рядах и получите сводную таблицу результатов по всем метрикам, постройте также графики прогнозов. 

# short_season

In [112]:
plot_ts(short_season)

Loading BokehJS ...

## Скользящие статистики

In [113]:
best_in_sample, best_out_of_sample = hyperparameters_search(
                                short_season,
                                'PT1H',
                                RollingEstimator,
                                24,
                                {'num_lags': [3, 12, 24, 48], 'model__rolling_filter': ['mean', 'median']},
                                verbose=False,
                                use_mappers=False
                            )

In [115]:
train, test, in_sample, _ = make_pipeline(short_season, 'PT1H', RollingEstimator, use_mappers=False, **best_in_sample)
plot_ts(short_season, in_sample)

train, test, _, out_of_sample = make_pipeline(short_season, 'PT1H', RollingEstimator, use_mappers=False, **best_out_of_sample)
plot_ts(short_season, out_of_sample)

Loading BokehJS ...

Loading BokehJS ...

In [118]:
best_in_sample, best_out_of_sample = hyperparameters_search(
                                short_season,
                                'PT1H',
                                ExponentialSmoothingEstimator,
                                24,
                                {'num_lags': [3, 12, 24, 48, 72], 'model__alpha_coef': [0.1, 0.5, 1]},
                                verbose=False,
                                use_mappers=False
                            )

In [119]:
train, test, in_sample, _ = make_pipeline(short_season, 'PT1H', ExponentialSmoothingEstimator, use_mappers=False, **best_in_sample)
plot_ts(short_season, in_sample)

train, test, _, out_of_sample = make_pipeline(short_season, 'PT1H', ExponentialSmoothingEstimator, use_mappers=False, **best_out_of_sample)
plot_ts(short_season, out_of_sample)

Loading BokehJS ...

Loading BokehJS ...

## Линейная регрессия

In [120]:
best_in_sample, best_out_of_sample = hyperparameters_search(
                                short_season,
                                'PT1H',
                                LinearRegression,
                                24,
                                {
                                    'model__normalize': [True, False],
                                    'num_lags': [12, 24, 48],
                                },
                                verbose=False
                            )

In [121]:
train, test, in_sample, _ = make_pipeline(short_season, 'PT1H', LinearRegression, **best_in_sample)
plot_ts(short_season, in_sample)

train, test, _, out_of_sample = make_pipeline(short_season, 'PT1H', LinearRegression, **best_out_of_sample)
plot_ts(short_season, out_of_sample)

Loading BokehJS ...

Loading BokehJS ...

In [122]:
best_in_sample, best_out_of_sample = hyperparameters_search(
                                short_season,
                                'PT1H',
                                Lasso,
                                24,
                                {'model__normalize': [True, False], 'num_lags': [12, 24, 48], 'model__alpha': [0, 0.01, 3, 10, 1000]},
                                verbose=False
                            )

In [123]:
train, test, in_sample, _ = make_pipeline(short_season, 'PT1H', Ridge, **best_in_sample)
plot_ts(short_season, in_sample)

train, test, _, out_of_sample = make_pipeline(short_season, 'PT1H', Ridge, **best_out_of_sample)
plot_ts(short_season, out_of_sample)

Loading BokehJS ...

Loading BokehJS ...

## ARIMA

In [82]:
train, test, in_sample, out_of_sample = make_pipeline_arima(short_season, period=24, seasonal=True, d=0, D=1)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,1,1)[24] intercept   : AIC=inf, Time=3.69 sec
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=2731.552, Time=0.03 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=2337.655, Time=0.90 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=1.67 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=2729.556, Time=0.03 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=2345.019, Time=0.12 sec
 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=2279.835, Time=4.15 sec
 ARIMA(1,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=7.74 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=2.40 sec
 ARIMA(0,0,0)(2,1,0)[24] intercept   : AIC=inf, Time=2.37 sec
 ARIMA(2,0,0)(2,1,0)[24] intercept   : AIC=2254.837, Time=4.13 sec
 ARIMA(2,0,0)(1,1,0)[24] intercept   : AIC=2283.168, Time=1.54 sec
 ARIMA(2,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=9.58 sec
 ARIMA(2,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=3.45 sec
 ARIMA(3,0,0)(2,1,0)[24] intercept   : AIC=2205.890, T

In [83]:
plot_ts(train, test, in_sample, out_of_sample)

Loading BokehJS ...

## RandomForest

In [91]:
param_grid = {
    'model__max_depth': [6, 12],
    'model__n_estimators': [50, 500, 1000],
    'num_lags': [24, 48]
}

best_in_sample, best_out_of_sample = hyperparameters_search(
                                        short_season,
                                        'PT1H',
                                        RandomForestRegressor,
                                        24,
                                        param_grid,
                                        verbose=False
                                    )

train, test, in_sample, _ = make_pipeline(
    short_season, 'PT1H', RandomForestRegressor, **best_in_sample)
plot_ts(train, test, in_sample)

train, test, _, out_of_sample = make_pipeline(
    short_season, 'PT1H', RandomForestRegressor, **best_out_of_sample)
plot_ts(train, test, out_of_sample)

Loading BokehJS ...

Loading BokehJS ...

## Gradient Boosting

In [116]:
param_grid = {
    'model__max_depth': [6, 12],
    'model__n_estimators': [50, 500, 1000],
    'num_lags': [24, 48]
}

best_in_sample, best_out_of_sample = hyperparameters_search(
                                        short_season,
                                        'PT1H',
                                        GradientBoostingRegressor,
                                        24,
                                        param_grid,
                                        verbose=False
                                    )

train, test, in_sample, _ = make_pipeline(
    short_season, 'PT1H', GradientBoostingRegressor, **best_in_sample)
plot_ts(train, test, in_sample)

train, test, _, out_of_sample = make_pipeline(
    short_season, 'PT1H', GradientBoostingRegressor, **best_out_of_sample)
plot_ts(train, test, out_of_sample)

Loading BokehJS ...

Loading BokehJS ...

### Выводы

Ряды с короткой историей.

* на рядах с короткой историей нет смысла использовать "деревянные" алгоритмы, вследствие трудностей с переобучением
* линейную регрессию можно использовать для прогноза на одну точку, нужно быть аккуратным для прогноза на много лагов
* хорошо подходит арима (нужно подбирать d, D); если у вас сложный кейс (полиномиальный тренд, мультипликативная сезонность) - настраивать ариму стоит вручную


Ряды с трендом.
* нельзя использовать "деревянные алгоритмы", т.к. они не умеют улавливать тренд
* линейная регрессия может быть использовать для прогноза на одну точку
* хорошо подходит arima

Интегрированные ряды (блуждания, акции и тп).
* либо наивное
* либо arima (которая выродится в наивное)

Зашумленные ряды.
* главное - не переобучить модель
* arima из коробки работает "почти хорошо"



Таким образом, arima почти везде показывает хорошо, однако у нее есть две проблемы:
* невозомжность автоматически сделать ряд стационарным
* нелинейный рост сложности алгоритма с ростом данных => проблемно использовать на длинных рядах или на рядах с большим количеством доп регрессоров (признаков)

Для предсказания на одну точку вперед можно построить хорошую модель гораздо проще, в частности линейная регрессия почти везде показывает себя хорошо. 


Деревянные модели показывают себя хорошо только на рядах с большим количеством данных.